In [1]:
# Download data fro3m http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
# %lprun -f

In [2]:
import numpy as np
import re
import collections
import operator
from scipy.optimize import minimize
import copy

from scipy.optimize import fmin_bfgs

In [3]:
def findDic(text):
    words = re.compile('[a-zA-Z\']*[a-zA-Z]').findall(text)
    return collections.Counter(words)

In [4]:
def readData(directory, paths, limit=None):
    if limit:
        paths = paths[0:limit]

    data = []
    for path in paths:
        txt = open(directory + path).read()
        data.append(txt)
    return data
        
        
negDataPath = !ls data/neg/
negData = readData('./data/neg/', negDataPath, 1000)

posDataPath = !ls data/pos/
posData = readData('./data/pos/', posDataPath, 1000)

In [5]:
negData = [
    'I am exhausted of this work.',
    "I can't cooperate with this",
    'I do not enjoy my job',
    "I feel brilliant!",
    'Gary is a friend of mine.',
    'The date was good.'
]
posData = [
    'I love this burger.',
    'This is an brilliant place!',
    'I feel very good about these dates.',
    'This is my best work.',
    "What an awesome view"
]

test = [
    "I am not feeling well today.",
    "I can't believe I'm doing this.",
    'The date was good.',
    "I am not feeling well today.",
    "I can't believe I'm doing this.",
    'I do not enjoy my job'
]

In [6]:
print negData[0]
print "+++++"
print posData[0]

I am exhausted of this work.
+++++
I love this burger.


In [7]:
def findWordCount(data):
    wordCount = []
    for review in data:
        wordCount.append(findDic(review))
    return wordCount

negDataDic = findWordCount(negData)
posDataDic = findWordCount(posData)

In [8]:
def updateDic(mainDic, dicArray):
    mainDic = copy.deepcopy(mainDic)
    for dic in dicArray:
        mainDic.update(dic)
    
    # update indexes
    for index, key in enumerate(mainDic):
        mainDic[ key ] = index
        
    return mainDic

allWords = {}
allWords = updateDic(allWords, negDataDic)
allWords = updateDic(allWords, posDataDic)

In [9]:
def findRow(words, text, boolean=False):
    row = np.zeros(len(allWords))
    wordsDic = findDic(text)
    for word in words:
        if word in allWords:
            row[allWords[word]] = 1 if boolean else wordsDic[word]
    return row

In [10]:
def createTable(allWords, negData, y0, posData, y1):
    x = []
    for data in negData + posData:
        x.append( findRow(allWords, data) )
    x = np.row_stack(x)
    
    y = np.append(y0 * np.ones(len(negData)), y1 * np.ones(len(posData)))
    return x, y

In [11]:
x, y = createTable(allWords, negData, -100, posData, 100)

In [12]:
def cost(w, b, c, x, y):
    p = 1 - y * (x.dot(w) + b)
    p[p<0] = 0
    p = p**2
    return w.dot(w.T) + c * p.sum()

In [13]:
posX = x[:len(posData)]
negX = x[len(posData):]

In [18]:
# x = np.array([[2,1], [2,-1], [4,4], [4, -4], [-2,1], [-2,-1], [-4,4], [-4, -4]])
# y = np.array([1,1,1,1,-1,-1,-1,-1])

In [1]:
M = len(allWords)
w = np.zeros(M)
b = 0
c = 100
 
Nfeval = 1
print '{0:s}  {1:s}'.format('iter', 'cost')
def callbackF(Xi):
    return
    global Nfeval
    global b, c, x, y
    print '{0:4d}  {1:f}'.format(Nfeval, cost(np.array(Xi), b, c, x, y))
    Nfeval += 1
    
res = minimize(cost, w, method='Nelder-Mead', args=(b, c, x, y), options={'xtol': 1e-8, 'disp': True}, callback=callbackF)
res.x

NameError: name 'allWords' is not defined

In [22]:
posX.dot(res.x), negX.dot(res.x)

ValueError: shapes (5,38) and (2,) not aligned: 38 (dim 1) != 2 (dim 0)

In [16]:
b = (min(posX.dot(res.x)) + max(negX.dot(res.x))) /2

In [183]:
b

-0.0069831569339007058

In [182]:
for r in test:
    print(findRow(allWords, r).dot(res.x))

-0.0201638554568
-0.0102811031689
-0.010925294643
-0.0201638554568
-0.0102811031689
-0.0231442454485


In [149]:
import numpy as np

In [150]:
from sklearn.svm import SVC

In [157]:
clf = SVC()
clf.fit(x, y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [158]:
for i in test:
    print( clf.predict( findRow(allWords, i) ) )

[-100.]
[-100.]
[-100.]
[-100.]
[-100.]
[-100.]


/Users/sadegh/.ml/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/sadegh/.ml/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/sadegh/.ml/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  Deprec

In [21]:
x[5]

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [16]:
y[-100:]

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

In [29]:
x[3].sum()

152.0